In [4]:
import json
import tensorflow as tf

In [5]:
# Load the questions and answers from file
with open('questions_answers.json') as f:
    data = json.load(f)
questions = [entry['question'] for entry in data]
answers = [entry['answer'] for entry in data]

# Create a set of all unique words in the questions and answers
vocab = set()
for question in questions + answers:
    vocab.update(question.split())

# Declare the vocabulary size
vocab_size = len(vocab)

# You can also use Tokenizer from tensorflow to fit the text on the vocabulary and get the vocab_size
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(questions + answers)
vocab_size = len(tokenizer.word_index) + 1



In [7]:
print(vocab_size)
print(answers)
print(questions)

34


In [6]:
# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((questions, answers))
dataset = dataset.shuffle(buffer_size=1024).batch(32)

# Create the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Fit the model
history = model.fit(dataset, epochs=10)


Epoch 1/10


ValueError: in user code:

    File "/Users/macos/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/macos/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/macos/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/macos/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/Users/macos/opt/anaconda3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/macos/opt/anaconda3/envs/tf/lib/python3.9/site-packages/

In [ ]:
# Test the model
test_question = "Berapa jumlah 1 + 1?"
predicted_answer = model.predict(test_question)
print("Answer:", predicted_answer)